###### References: 
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python, 2nd Edition, by Luciano Ramalho. Chapter 8: Type Hints in Functions

# Gradual Typing
PEP-484 https://peps.python.org/pep-0484/

- Is optional
- Does not catch type errors at runtime
- Does not enhance performance

### Python Type Checkers:
- Mypy
- Google's pytype
- Microsoft's Pyright
- Facebook's Pyre
- PyCharm

In [1]:
def show_count(count, word):
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'

In [2]:
show_count(99, 'bird')

'99 birds'

In [3]:
show_count(1, 'bird')

'1 bird'

In [4]:
!mypy messages_0.py

Success: no issues found in 1 source file


#### Starting with Mypy

In [5]:
from pytest import mark

In [6]:
@mark.parameterize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts'),
])
def test_show_count(qty, expected):
    got = show_count(qty, 'part')
    assert got == expected

In [7]:
def test_show_count_zero():
    got = show_count(0, 'part')
    assert got == 'no parts'

In [8]:
!mypy --disallow-untyped-defs messages_test_0.py

messages_0.py:14: error: Function is missing a type annotation  [no-untyped-def]
messages_test_0.py:9: error: Function is missing a type annotation  [no-untyped-def]
messages_test_0.py:13: error: Function is missing a return type annotation  [no-untyped-def]
messages_test_0.py:13: note: Use "-> None" if function does not return a value
Found 3 errors in 2 files (checked 1 source file)


In [9]:
!mypy --disallow-incomplete-defs messages_test_0.py

Success: no issues found in 1 source file


In [10]:
def show_count(count: int, word: str) -> str:
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'

In [11]:
!mypy --disallow-untyped-defs messages_1.py

Success: no issues found in 1 source file


## A Default Parameter Value

In [12]:
def show_count(count: int, singular: str, plural: str = '') -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'

In [13]:
show_count(3, 'mouse', 'mice')

'3 mice'

In [14]:
!mypy --disallow-untyped-defs messages_2.py

Success: no issues found in 1 source file


In [15]:
from typing import Optional

In [16]:
def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'

## Types are Defined by Supported Operations

In [17]:
def double(x):
    return x * 2

In [18]:
from collections import abc

In [19]:
def double(x: abc.Sequence):
    return x * 2

- Duck Typing
    - Objects have types, but variables are untyped.
- Nominal Typing
    - Objects and variables have types. But objects only exist at runtime, and type checker only ares about the source code whree variables are annotated with type hints.

In [20]:
class Bird:
    pass

class Duck(Bird):  # <1>
    def quack(self):
        print('Quack!')

def alert(birdie):  # <2>
    birdie.quack()

def alert_duck(birdie: Duck) -> None:  # <3>
    birdie.quack()

def alert_bird(birdie: Bird) -> None:  # <4>
    birdie.quack()

In [21]:
!mypy birds.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


In [22]:
daffy = Duck()
alert(daffy)       # <1>
alert_duck(daffy)  # <2>
alert_bird(daffy)

Quack!
Quack!
Quack!


In [23]:
!mypy daffy.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


In [24]:
!mypy woody.py

birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
woody.py:5: error: Argument 1 to "alert_duck" has incompatible type "Bird"; expected "Duck"  [arg-type]
Found 2 errors in 2 files (checked 1 source file)


In [25]:
woody = Bird()
alert(woody)
alert_duck(woody)
alert_bird(woody)

AttributeError: 'Bird' object has no attribute 'quack'

## Types Usable in Annotations

### The Any type
`typing.Any`,
dynamic type

In [26]:
def double(x):
    return x * 2

assumes:

In [27]:
from typing import Any

In [28]:
def double(x: Any) -> Any:
    return x * 2

as opposed to:

In [29]:
def double(x: object) -> object:
    return x * 2

In [30]:
!mypy double_object.py

double_object.py:2: error: Unsupported operand types for * ("object" and "int")  [operator]
Found 1 error in 1 file (checked 1 source file)


### Subtype-`of` versus consistent-`with`

### Simple Types and Classes

### Optional and Union Types

In [31]:
def show_count(count: int, singular: str, plural: str | None = None) -> str:
    pass

In [32]:
from typing import Union

In [33]:
def parse_token(token: str) -> Union[str, float]:
    try:
        return float(token)
    except ValueError:
        return token

### Generic Collections

In [34]:
def tokenize(text: str) -> list[str]:
    return txt.upper().split()

### Tuple Types
- Tuples as records
- Tuples as records with named fields
- Tuples as immutable sequences

#### Tuples as records

In [36]:
from geolib import geohash as gh  # type: ignore # stops 

In [37]:
PRECISION = 9

In [38]:
def geohash(lat_lon: tuple[float, float]) -> str:  # <2>
    return gh.encode(*lat_lon, PRECISION)

In [39]:
shanghai = 31.2304, 121.4737

In [40]:
geohash(shanghai)

'wtw3sjq6q'

#### Tuples as records with named fields

In [41]:
from typing import NamedTuple

In [42]:
class Coordinate(NamedTuple):
    lat: float
    lon: float

def geohash(lat_lon: Coordinate) -> str:
    return gh.encode(*lat_lon, PRECISION)

In [43]:
def display(lat_lon: tuple[float, float]) -> str:
    lat, lon = lat_lon
    ns = 'N' if lat >= 0 else 'S'
    ew = 'E' if lon >= 0 else 'W'
    return f'{abs(lat):0.1f}°{ns}, {abs(lon):0.1f}°{ew}'

In [44]:
def demo():
    shanghai = 31.2304, 121.4737
    print(display(shanghai))
    s = geohash(shanghai)
    print(s)

In [45]:
demo()

31.2°N, 121.5°E
wtw3sjq6q


#### Tuples as immutable sequences

In [46]:
from collections.abc import Sequence

In [47]:
def columnize(
    sequence: Sequence[str], num_columns: int = 0
) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, reminder = divmod(len(sequence), num_columns)
    num_rows += bool(reminder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

In [48]:
animals = 'drake fawn heron ibex koala lynx lahr xerus yak zapus'.split()

In [49]:
table = columnize(animals)
table

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'lahr'),
 ('ibex', 'xerus')]

#### Generic Mappings

In [50]:
import sys
import re
import unicodedata
from collections.abc import Iterator

In [51]:
RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

In [52]:
def tokenize(text: str) -> Iterator[str]:  # <1>
    """return iterable of uppercased words"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()

In [53]:
def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    index: dict[str, set[str]] = {}  # <2>
    for char in (chr(i) for i in range(start, end)):
        if name := unicodedata.name(char, ''):  # <3>
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)
    return index

In [54]:
index = name_index(32, 65)

In [55]:
sorted(index['SIGN'])

['#', '$', '%', '+', '<', '=', '>']

In [56]:
sorted(index['DIGIT'])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [57]:
index['DIGIT'] & index['EIGHT']

{'8'}

#### Abstract Base Classes

_Be conservative in what you send, be liberal in what you accept._ -- Postel's law a.k.a. the Robustness Principle

    from collections.abc import Mapping
    
    def name2hex(name: str, color_map: Mapping[str, int]) -> str:
    
versus

    def name2hex(name: str, color_map: dict[str, int]) -> str:

#### The fall of the numeric tower

PEP 3141 -- A Type Hierarchy for Numbers.
- Number
- Complex
- Real
- Rational
- Integral

#### Iterable
`typing.list`

    def fsum(__seq: Iterable[float]) -> float:

In [58]:
from collections.abc import Iterable

In [59]:
FromTo = tuple[str, str]

In [60]:
def zip_replace(text: str, changes: Iterable[FromTo]) -> str:  # <2>
    for from_, to in changes:
        text = text.replace(from_, to)
    return text

#### (`abc.Iterable` versus `abc.Sequence`)
Chapter 17

#### Parameterized Generic and TypeVar

In [61]:
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

In [62]:
T = TypeVar('T')

In [63]:
def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]

In [64]:
def demo() -> None:
    import typing
    p1 = tuple(range(10))
    s1 = sample(p1, 3)
    if typing.TYPE_CHECKING:
        reveal_type(p1)
        reveal_type(s1)
    print(p1)
    print(s1)
    p2 = 'The quick brown fox jumps over the lazy dog'
    s2 = sample(p2, 10)
    if typing.TYPE_CHECKING:
        reveal_type(p2)
        reveal_type(s2)
    print(p2)
    print(s2)

In [65]:
demo()

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
[8, 4, 1]
The quick brown fox jumps over the lazy dog
['u', 'e', 'm', 'b', ' ', 'g', 'z', ' ', 'e', 'f']


##### Problematic way of coding:

In [66]:
from collections import Counter
from collections.abc import Iterable

In [67]:
def mode(data: Iterable[float]) -> float:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

##### Problem occurs when TypeVar appears the second time. As it will mean the same type as the first
    from collections.abc import Iterable
    from typing import TypeVar
    
    T = TypeVar('T')
    
    def mode(data: Iterable[T]) -> T:

#### Restricted TypeVar

Set one of the types named in the TypeVar declaration.

    from collections.abc import Iterable
    from decimal import Decimal
    from fractions import Fraction
    from typing import TypeVar
    
    NumberT = TypeVar('NumberT', float, Decimal, Fraction)
    
    def mode(data: Iterable[NumberT]) -> NumberT:
    
#### Bonded TypeVar
Set to an inferred type of the expression -- as long as the infered type of _consistent-with_ the __boundary__ declared in the `bound=` keyword argument of TypeVar.

In [68]:
from collections import Counter
from collections.abc import Iterable, Hashable
from typing import TypeVar

In [69]:
HashableT = TypeVar('HashableT', bound=Hashable)

In [70]:
def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

#### The AnyStr predefined type variable

    AnyStr = TypeVar('AnyStr', bytes, str)
    
### Static Protocols
PEP 544 -- Protocols: Structural subtyping

In [71]:
def top(series: Iterable[T], length: int) -> list[T]:
    ordered = sorted(series, reverse = True)
    return ordered[:length]

In [72]:
from typing import Protocol, Any

In [73]:
class SupportsLessThan(Protocol):  # <1>
    def __lt__(self, other: Any) -> bool: ...

In [74]:
from collections.abc import Iterable
from typing import TypeVar

In [75]:
LT = TypeVar('LT', bound=SupportsLessThan)

In [76]:
def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [77]:
from collections.abc import Iterator
from typing import TYPE_CHECKING  # <1>

import pytest

In [79]:

@pytest.mark.parametrize('series, length, expected', [
    ((1, 2, 3), 2, [3, 2]),
    ((1, 2, 3), 3, [3, 2, 1]),
    ((3, 3, 3), 1, [3]),
])
def test_top(
    series: tuple[float, ...],
    length: int,
    expected: list[float],
) -> None:
    result = top(series, length)
    assert expected == result

In [80]:
def test_top_tuples() -> None:
    fruit = 'mango pear apple kiwi banana'.split()
    series: Iterator[tuple[int, str]] = (  # <2>
        (len(s), s) for s in fruit)
    length = 3
    expected = [(6, 'banana'), (5, 'mango'), (5, 'apple')]
    result = top(series, length)
    if TYPE_CHECKING:  # <3>
        reveal_type(series)  # <4>
        reveal_type(expected)
        reveal_type(result)
    assert result == expected

In [81]:
# intentional type error
def test_top_objects_error() -> None:
    series = [object() for _ in range(4)]
    if TYPE_CHECKING:
        reveal_type(series)
    with pytest.raises(TypeError) as excinfo:
        top(series, 3)  # <5>
    assert "'<' not supported" in str(excinfo.value)

### Callable
    Callable[[ParamType1, ParamType2], ReturnType]
    
E.g. Pattern Matching in lis.py

    def repl(input_fn: Callable[[Any], str] = input]) -> None:
    
E.g. `input()`

    def input(__promp: Any = ...) -> str: ...
    
#### Variances in Callable Types

In [82]:
from collections.abc import Callable

In [83]:
def update(
        probe: Callable[[], float],
        display: Callable[[float], None]
    ) -> None:
    temperature = probe()
    #imagine lots of control code here
    display(temperature)
                          
def probe_ok() -> int:
    return 42

def display_wrong(temperature: int) -> None:
    print(hex(temperature))

update(probe_ok, display_wrong)

0x2a


In [84]:
def display_ok(temperature: complex) -> None:
    print(temperature)

In [85]:
update(probe_ok, display_ok)

42


#### NoReturn
    def exit(__status: object = ...) -> NoReturn: ...
    
### Annotating Positional Only and Variadic Parameters
    def tag(name, /, *content, class_=None, **attrs):

In [86]:
def tag(
    name: str,
    /,
    *content: str,
    class_: Optional[str] = None,
    **attr: str,
) -> str:
    pass

### Imperfect Typing and Strong Testing
- False positives
    - Tools report type errors on code that is correct
- False negatives
    - Tools don't report type errors on code that is incorrect
    


- some handy features cannot be statically checked
    `config(**settings)`
- Advanced features like properies, descriptors, metaclasses, and metaprogramming, are poorly supported.

- Type checkers lag behind Python releases